In [13]:
from test_data_generator import generate_fake_metro, plot_metro_layout
import random
import pandas as pd

flow_paths, stations, df = generate_fake_metro(
    station_count=10,
    flow_path_count=10,
    max_flow_path_length=5,
    min_flow_path_frequency=1,
    max_flow_path_frequency=10
)

test_stations = stations
test_paths = flow_paths

In [8]:
def generate_slots(test_paths):
    """
    Generates slots given the paths
    """
    max_length = int(len(test_paths) / 2) + 1
    list_of_list = [[(i,i), (i, -i), (-i, i), (-i, -i)] for i in range (1, max_length)]

    flattened_list = [(0, 0)]

    for sublist in list_of_list:
        for tuple_item in sublist:
            flattened_list.append(tuple_item)
    slot_names = ['s{}'.format(i) for i in range(0, len(flattened_list))]
    return list(zip(slot_names, flattened_list))


In [105]:
from algo import combination_to_coordinates, count_intersections
from utils import Point, LayoutAlgorithm, FlowPathsT, LayoutOutput, SLOTS

def calculate_intersections(flow_paths, stations):
    """
    Takes a station, the flowpaths with frequencies and computes the optimal arrangement
    """

    slots = generate_slots(test_paths)

    data_paths = {
    'Paths': [n[1] for n in flow_paths],
    'Frequency': [n[0] for n in flow_paths],
    'Slot offsets': random.sample(slots, 10)
    }

    df_paths = pd.DataFrame(data_paths)

    # Transform offsets to a dictionary
    OFFSETS = df_paths['Slot offsets']
    offset_dict = dict()
    for offset in OFFSETS:
        offset_dict.update({offset[0]: offset[1]})

    # Get slot names
    slot_names = list(offset_dict.keys())

    SLOT_OFFSETS = offset_dict
    SLOTS = slot_names

    # Get slot names
    slot_coordinates = {}

    #Transforms the slot to a location
    for station_name, point in stations.items():
        for slot in SLOTS:
            offset_x, offset_y = SLOT_OFFSETS[slot]
            slot_coordinates[(station_name, slot)] = Point(point.x + offset_x, point.y + offset_y)

    combination_merged = []

    # For each path in the dataframe, it takes the slot offset and takes the name.
    # Then for each element of the path it creates a tuple with both.
    for path in df_paths.iterrows():
        path_list = []
        slot = path[1]['Slot offsets'][0]
        for station in path[1]['Paths']:
            path_list.append((station, slot))
        combination_merged.append(path_list)

    intersections = count_intersections(combination_to_coordinates(combination_merged, slot_coordinates))
    layout = list(map(lambda x: (1, x), combination_to_coordinates(combination_merged, slot_coordinates)))
    return intersections, df_paths, layout

intersections_test , paths_dummy , layout= calculate_intersections(test_paths, test_stations)
intersections_test

38

In [71]:
def dynamic_ranges(self, flow_paths, stations):
    """
    Takes the coordinates of a station, the paths between computes the optimal arrangement
    """
    #TODO: keep track of used combinations
    #TODO: make the iteration dependend on the search space

    intersections, df_paths = calculate_intersections(flow_paths, stations)
    return intersections, df_paths

dynamic_ranges([], test_paths, test_stations)

(47,
                    Paths  Frequency     Slot offsets
 0  [Root, G, I, B, J, E]          9    (s2, (1, -1))
 1        [Root, G, I, B]          8   (s15, (-4, 4))
 2        [Root, G, I, B]         10     (s5, (2, 2))
 3              [Root, F]          5    (s6, (2, -2))
 4  [Root, F, E, J, B, I]          5   (s19, (-5, 5))
 5              [Root, G]          2   (s4, (-1, -1))
 6     [Root, F, E, J, B]          8  (s20, (-5, -5))
 7     [Root, G, I, B, J]          5    (s13, (4, 4))
 8           [Root, G, I]          5  (s16, (-4, -4))
 9     [Root, G, I, B, J]          5     (s1, (1, 1)))

In [11]:
def create_combination(paths, slots):
    """
    Creates a random combination of order and slots with the paths
    """

    combination = []

    # Pick a random path and a random slot and add it to combinations
    for c in range(len(paths)):
        path = random.choice(paths)
        paths.remove(path)
        slot = random.choice(slots)
        slots.remove(slot)
        combination.append([(station, slot) for station in path[1]])

    return combination


def dynamic_ranges(self, flow_paths, stations):
    """
    Takes the coordinates of a station, the paths between computes the optimal arrangement
    """
    #TODO: keep track of used combinations
    #TODO: make the iteration dependend on the search space

    slots = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10']
    paths = flow_paths
    # Initialize iteration counter
    iteration = 0
    
    # Initialize dictionary with lowest scores
    amount_of_lower_scores = 100
    lowest_scores = {key: 1000 for key in range(amount_of_lower_scores)}

    # Initialize score ranges
    score_range = 0

    # Initialize decreasing rates
    rate_lowest_scores = 10
    rate_score_range = 10

    # Compute score range
    search_space = len(slots) * len(paths) * len(paths)

    # Find the lowest score while ranges don't match
    while lowest_scores[max(lowest_scores, key = lowest_scores.get)] > score_range and len(lowest_scores) > 1:

        combinations = create_combination(paths, slots)

        #TODO: map back to locations using stations
        # Calculate intersections
        # intersection_value = calculate_intersections(flow_paths, stations)
        intersection_value = 10
        # Calculate overlap

        if intersection_value < max(lowest_scores.values()):
            key = ""
            for combo in combinations:
                combo = dict(combo)
                key = "".join(combo.keys()) + list(combo.values())[0] + key
            lowest_scores.update({key: intersection_value})

        if iteration % rate_lowest_scores == 0:
            lowest_scores.pop(max(lowest_scores, key = lowest_scores.get))

        if iteration % rate_score_range == 0:
            score_range += 1

        iteration += 1



    return min(lowest_scores, key = lowest_scores.get), lowest_scores[min(lowest_scores, key = lowest_scores.get)]

dynamic_ranges([], test_paths, test_stations)

('RootJs1RootJs7RootJHIEs8RootJCGDAs4RootJCGDs2RootJCGs5RootJHIs9RootJCGDs10RootJCGDs6RootJHIs3',
 10)